In [2]:
import sys
sys.path.append('/kaggle/working/RL-code/baseline')
import baseline_methods as bm
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

In [1]:
!git clone https://github.com/TobyYang7/RL-code

Cloning into 'RL-code'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 129 (delta 11), reused 28 (delta 4), pack-reused 94
Receiving objects: 100% (129/129), 99.70 MiB | 18.45 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (85/85), done.


In [3]:
us_delay = np.load('/kaggle/working/RL-code/baseline/udata/udelay.npy')
print(us_delay.shape)
# Shape(N, T, F)

# us_delay_1 arrival delay
us_delay_1 = us_delay[:, :, 0]
# us_delay_1 departure delay
us_delay_2 = us_delay[:, :, 1]

print(us_delay_1.shape)

(70, 78912, 2)
(70, 78912)


# HA


In [4]:
# y_predict, y_test = bm.historical_average_predict(
#     cn_delay_1, period=2*18, test_ratio=0.2)
# ha_a_mae, ha_a_rmse, ha_a_mape = bm.test_error(y_predict, y_test)
# y_predict, y_test = bm.historical_average_predict(
#     cn_delay_2, period=2*18, test_ratio=0.2)

y_predict, y_test = bm.historical_average_predict(
    us_delay_1, period=2*18, test_ratio=0.2)
ha_a_mae, ha_a_rmse, ha_a_mape = bm.test_error(y_predict, y_test)
y_predict, y_test = bm.historical_average_predict(
    us_delay_2, period=2*18, test_ratio=0.2)
ha_d_mae, ha_d_rmse, ha_d_mape = bm.test_error(y_predict, y_test)
print(
    f"The test error for arrival prediction delay using HA is {round(ha_a_mae,3), round(ha_a_rmse,3), round(ha_a_mape,3)}")
print(
    f"The test error for delay prediction delay using HA is {round(ha_d_mae,3), round(ha_d_rmse,3), round(ha_d_mape,3)}")

/kaggle/working/RL-code/baseline/baseline_methods.py:73: RuntimeWarning: Mean of empty slice.
  y_predict[k, i - n_train] = historical[k, :][~np.isnan(historical[k, :])].mean()
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


The test error for arrival prediction delay using HA is (9.089, 11.847, 1.149)
The test error for delay prediction delay using HA is (6.519, 8.631, 1.284)


# VAR


In [6]:
time = [2, 5, 11]

In [7]:
# y_predict_a, y_test_a = bm.var_predict(cn_delay_1, test_ratio=0.2)
# y_predict_d, y_test_d = bm.var_predict(cn_delay_2, test_ratio=0.2)
y_predict_a, y_test_a = bm.var_predict(us_delay_1, test_ratio=0.2)
y_predict_d, y_test_d = bm.var_predict(us_delay_2, test_ratio=0.2)
var_a_mae = []
var_a_rmse = []
var_a_mape = []
var_d_mae = []
var_d_rmse = []
var_d_mape = []
for i in time:
    a, b, c = bm.test_error(y_predict_a[i, :, :].T, y_test_a)
    var_a_mae.append(round(a, 3))
    var_a_rmse.append(round(b, 3))
    var_a_mape.append(round(c, 3))
    a, b, c = bm.test_error(y_predict_d[i, :, :].T, y_test_d)
    var_d_mae.append(round(a, 3))
    var_d_rmse.append(round(b, 3))
    var_d_mape.append(round(c, 3))

for i, x in enumerate(zip(var_a_mae, var_a_rmse, var_a_mape)):
    print(f"Error of VAR in {time[i]+1}-step arrival: {x}")

for i, x in enumerate(zip(var_d_mae, var_d_rmse, var_d_mape)):
    print(f"Error of VAR in {time[i]+1}-step departure: {x}")

Error of VAR in 3-step arrival: (7.795, 10.468, 1.211)

Error of VAR in 6-step arrival: (8.123, 10.824, 1.224)

Error of VAR in 12-step arrival: (8.479, 11.237, 1.216)

Error of VAR in 3-step departure: (5.561, 7.656, 1.129)

Error of VAR in 6-step departure: (5.817, 7.925, 1.14)

Error of VAR in 12-step departure: (6.165, 8.304, 1.129)


# ARIMA


In [8]:
# y_predict_a, y_test_a = bm.arima_predict(cn_delay_1, test_ratio=0.02)
# y_predict_d, y_test_d = bm.arima_predict(cn_delay_2, test_ratio=0.02)

y_predict_a, y_test_a = bm.arima_predict(us_delay_1, test_ratio=0.02)
y_predict_d, y_test_d = bm.arima_predict(us_delay_2, test_ratio=0.02)

In [9]:
print(np.sum(np.isnan(y_predict_a)))
print(y_predict_a.shape)
print(y_test_a.shape)

0

(12, 1578, 70)

(70, 1578)


In [10]:
arima_a_mae = []
arima_a_rmse = []
arima_a_mape = []
arima_d_mae = []
arima_d_rmse = []
arima_d_mape = []
for i in time:
    a, b, c = bm.test_error(y_predict_a[i, :, :].T, y_test_a)
    arima_a_mae.append(round(a, 3))
    arima_a_rmse.append(round(b, 3))
    arima_a_mape.append(round(c, 3))
    a, b, c = bm.test_error(y_predict_d[i, :, :].T, y_test_d)
    arima_d_mae.append(round(a, 3))
    arima_d_rmse.append(round(b, 3))
    arima_d_mape.append(round(c, 3))

for i, x in enumerate(zip(arima_a_mae, arima_a_rmse, arima_a_mape)):
    print(f"Error of ARIMA in {time[i]+1}-step arrival: {x}")

for i, x in enumerate(zip(arima_d_mae, arima_d_rmse, arima_d_mape)):
    print(f"Error of ARIMA in {time[i]+1}-step departure: {x}")

Error of ARIMA in 3-step arrival: (10.508, 13.894, 2.436)

Error of ARIMA in 6-step arrival: (10.481, 13.863, 2.419)

Error of ARIMA in 12-step arrival: (10.599, 14.02, 2.48)

Error of ARIMA in 3-step departure: (7.607, 10.549, 1.133)

Error of ARIMA in 6-step departure: (7.587, 10.551, 1.123)

Error of ARIMA in 12-step departure: (7.653, 10.643, 1.142)


# SVM


In [ ]:
# y_predict_a, y_test_a = bm.var_predict_svr(cn_delay_1, test_ratio=0.2)
# y_predict_d, y_test_d = bm.var_predict_svr(cn_delay_2, test_ratio=0.2)
y_predict_a, y_test_a = bm.var_predict_svr(us_delay_1, test_ratio=0.2)
y_predict_d, y_test_d = bm.var_predict_svr(us_delay_2, test_ratio=0.2)
np.savez("tmp_svr.npz", y_predict_a=y_predict_a, y_test_a=y_test_a,
         y_predict_d=y_predict_d, y_test_d=y_test_d)

  0%|          | 0/70 [00:00<?, ?it/s]

In [ ]:
# y_test_a = y_test_a[:-36]
time = [2, 5, 11]
out_len = 12
var_a_mae = []
var_a_rmse = []
var_a_mape = []
var_d_mae = []
var_d_rmse = []
var_d_mape = []

for i in time:
    shift = i+1
    num_sample = y_test_a.shape[-1]
    a, b, c = bm.test_error(
        y_predict_a[:, :, i], y_test_a[:, shift:num_sample-(out_len-shift)])
    # y_predict_a: [T, N]  y_test_a: [T, N], shift `i+1` slots
    var_a_mae.append(round(a, 3))
    var_a_rmse.append(round(b, 3))
    var_a_mape.append(round(c, 3))
    a, b, c = bm.test_error(
        y_predict_d[:, :, i], y_test_d[:, shift:num_sample-(out_len-shift)])
    # y_predict_a: [T, N]  y_test_a: [T, N], shift `i+1` slots
    var_d_mae.append(round(a, 3))
    var_d_rmse.append(round(b, 3))
    var_d_mape.append(round(c, 3))

for i, x in enumerate(zip(var_a_mae, var_a_rmse, var_a_mape)):
    print(f"Error of SVM in {(time[i]+1)}-step arrival: {x}")

for i, x in enumerate(zip(var_d_mae, var_d_rmse, var_d_mape)):
    print(f"Error of SVM in {(time[i]+1)}-step departure: {x}")